In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import numpy as np
import pandas as pd
from scipy.stats import t as tstat
from statsmodels.tools.tools import *

from optimize import opt 
from sampler import *
from algorithms import *

In [26]:
def t_test(result_list, T, delta=0.1):
    trial = len(result_list)
        
    pval_list = []
    
    for i in range(trial):        
        result = np.array(result_list[i])

        idx =np.array([i+1 for i in range(len(result))])
        s = np.sqrt(result[:, 1]/idx)
        thetas = result[:, 0]
        
        tt = thetas/s
        
        DF = len(thetas) - 1
                
        pval_list.append(tstat.sf(np.abs(tt), DF)*2)
    
    return np.array(pval_list)

In [27]:
def rejection_or_not(pval_list, alpha, size=-250):
    trial = len(pval_list)
    
    reject = 0
    
    for i in range(trial):
        if pval_list[i, size] < alpha:
            reject += 1
    
    reject /= trial
    
    return reject

In [28]:
def multiple_testing(pval_list, alpha):
    trial = len(pval_list)
    
    reject = 0
    
    for i in range(trial):
        if pval_list[i, -350] < alpha:
            reject += 150
        elif pval_list[i, -250]*2 < alpha:
            reject += 250
        elif pval_list[i, -150]*3 < alpha:
            reject += 350
        elif pval_list[i, -50]*4 < alpha:
            reject += 450
        else:
            reject += 500
        
    reject /= trial
    
    return reject

In [29]:
def sequential_testing(result_list, T, delta=0.1):
    trial = len(result_list)
    
    stopping_time = 0
    
    for i in range(trial):
        result = np.array(result_list[i])

        idx =np.array([i+1 for i in range(len(result))])
        v = result[:, 2]*idx
        thetas = np.abs(result[:, 0]*idx)
        lil = 1.1*(np.log(1/delta) + np.sqrt(2*v*np.log(np.log(v)/delta)))

        stop_times = idx[(thetas > lil) == True]

        if len(stop_times) > 0:
            stopping_time += idx[(thetas > lil) == True][0]

        else:
            stopping_time += T
        
    stopping_time /= trial
    
    return stopping_time

In [32]:
std0 = 0.
std1 = 0.

scenario = 'A'

trial = 1000
T = 500

In [33]:
rct_res = []
a2ipw_knn_res = []
a2ipw_nw_res = []
ma2ipw_knn_res = []
ma2ipw_nw_res = []
adaipw_knn_res =  []
adaipw_nw_res = []
dm_knn_res = []
dm_nw_res = []
hahn50_knn_res =  []
hahn50_nw_res =  []
hahn100_knn_res =  []
hahn100_nw_res =  []
true_res = []

for t in range(trial):
    print(t)
    
    rct = RCT()
    a2ipw_knn = A2IPW(method='Knn')
    a2ipw_nw = A2IPW(method='NW')
    ma2ipw_knn = MA2IPW(method='Knn')
    ma2ipw_nw = MA2IPW(method='NW')
    adaipw_knn = AdaIPW(method='Knn')
    adaipw_nw = AdaIPW(method='NW')
    dm_knn = DM(method='Knn')
    dm_nw = DM(method='NW')
    hahn50_knn = Hahn(method='Knn', first_phase=50)
    hahn50_nw = Hahn(method='NW', first_phase=50)
    hahn100_knn = Hahn(method='Knn', first_phase=100)
    hahn100_nw = Hahn(method='NW', first_phase=100)

    rct_temp = []
    a2ipw_knn_temp = []
    a2ipw_nw_temp = []
    ma2ipw_knn_temp = []
    ma2ipw_nw_temp = []
    adaipw_knn_temp =  []
    adaipw_nw_temp = []
    dm_knn_temp = []
    dm_nw_temp = []
    hahn50_knn_temp =  []
    hahn50_nw_temp =  []
    hahn100_knn_temp =  []
    hahn100_nw_temp =  []
    
    if scenario == 'A':
        Y_pop, T_pop, X_pop, true_ITE = dgps.ihdp_surface_A()
    elif scenario == 'B':
        Y_pop, T_pop, X_pop, true_ITE = dgps.ihdp_surface_B()

    perm = np.random.permutation(len(Y_pop))
    Y_pop = Y_pop[perm]
    X_pop = X_pop[perm]

    Y1_pop = Y_pop.copy()
    Y1_pop[T_pop==0] += true_ITE[T_pop==0]
    Y1_pop += np.random.normal(0, std1, size=len(Y1_pop))

    Y0_pop = Y_pop.copy()
    Y0_pop[T_pop==1] -= true_ITE[T_pop==1]
    Y0_pop += np.random.normal(0, std0, size=len(Y0_pop))
        
    for period_t in range(T):
        X = X_pop[period_t]
        Y1 = Y1_pop[period_t]
        Y0 = Y0_pop[period_t]
                
        rct(period_t, X, Y0, Y1)
        a2ipw_knn(period_t, X, Y0, Y1)
        a2ipw_nw(period_t, X, Y0, Y1)
        ma2ipw_knn(period_t, X, Y0, Y1)
        ma2ipw_nw(period_t, X, Y0, Y1)
        adaipw_knn(period_t, X, Y0, Y1)
        adaipw_nw(period_t, X, Y0, Y1)
        dm_knn(period_t, X, Y0, Y1)
        dm_nw(period_t, X, Y0, Y1)
        hahn50_knn(period_t, X, Y0, Y1)
        hahn50_nw(period_t, X, Y0, Y1)
        hahn100_knn(period_t, X, Y0, Y1)
        hahn100_nw(period_t, X, Y0, Y1)

        if period_t > 2:
            rct_temp.append(rct.effect())
            a2ipw_knn_temp.append(a2ipw_knn.effect())
            a2ipw_nw_temp.append(a2ipw_nw.effect())
            ma2ipw_knn_temp.append(ma2ipw_knn.effect())
            ma2ipw_nw_temp.append(ma2ipw_nw.effect())
            adaipw_knn_temp.append(adaipw_knn.effect())
            adaipw_nw_temp.append(adaipw_nw.effect())
            dm_knn_temp.append(dm_knn.effect())
            dm_nw_temp.append(dm_nw.effect())
            hahn50_knn_temp.append(hahn50_knn.effect())
            hahn50_nw_temp.append(hahn50_nw.effect())
            hahn100_knn_temp.append(hahn100_knn.effect())
            hahn100_nw_temp.append(hahn100_nw.effect())

    rct_res.append(rct_temp)
    a2ipw_knn_res.append(a2ipw_knn_temp)
    a2ipw_nw_res.append(a2ipw_nw_temp)
    ma2ipw_knn_res.append(ma2ipw_knn_temp)
    ma2ipw_nw_res.append(ma2ipw_nw_temp)
    adaipw_knn_res.append(adaipw_knn_temp)
    adaipw_nw_res.append(adaipw_nw_temp)
    dm_knn_res.append(dm_knn_temp)
    dm_nw_res.append(dm_nw_temp)
    hahn50_knn_res.append(hahn50_knn_temp)
    hahn50_nw_res.append(hahn50_nw_temp)
    hahn100_knn_res.append(hahn100_knn_temp)
    hahn100_nw_res.append(hahn100_nw_temp)
    true_res.append(true_ITE)
    
rct_res = np.array(rct_res)
a2ipw_knn_res = np.array(a2ipw_knn_res)
a2ipw_nw_res = np.array(a2ipw_nw_res)
ma2ipw_knn_res = np.array(ma2ipw_knn_res)
ma2ipw_nw_res = np.array(ma2ipw_nw_res)
adaipw_knn_res =  np.array(adaipw_knn_res)
adaipw_nw_res = np.array(adaipw_nw_res)
dm_knn_res = np.array(dm_knn_res)
dm_nw_res = np.array(dm_nw_res)
hahn50_knn_res =  np.array(hahn50_knn_res)
hahn50_nw_res =  np.array(hahn50_nw_res)
hahn100_knn_res =  np.array(hahn100_knn_res)
hahn100_nw_res =  np.array(hahn100_nw_res)
true_res = np.array(true_res)

0


/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1


/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


2


...

/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


998


/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


999


/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [34]:
delta0 = 0.1
mu = 4

MSEs = np.zeros((13, T-3))
STDs = np.zeros((13, T-3))
tau_time = np.zeros(13)
bf_time = np.zeros(13)

rejection250_time = np.zeros(13)
rejection500_time = np.zeros(13)

MSEs[0] = np.mean((rct_res[:, :, 0] - mu)**2, axis=0)
MSEs[1] = np.mean((a2ipw_knn_res[:, :, 0] - mu)**2, axis=0)
MSEs[2] = np.mean((a2ipw_nw_res[:, :, 0] - mu)**2, axis=0)
MSEs[3] = np.mean((ma2ipw_knn_res[:, :, 0] - mu)**2, axis=0)
MSEs[4] = np.mean((ma2ipw_nw_res[:, :, 0] - mu)**2, axis=0)
MSEs[5] = np.mean((adaipw_knn_res[:, :, 0] - mu)**2, axis=0)
MSEs[6] = np.mean((adaipw_nw_res[:, :, 0] - mu)**2, axis=0)
MSEs[7] = np.mean((dm_knn_res[:, :, 0] - mu)**2, axis=0)
MSEs[8] = np.mean((dm_nw_res[:, :, 0] - mu)**2, axis=0)
MSEs[9] = np.mean((hahn50_knn_res[:, :, 0] - mu)**2, axis=0)
MSEs[10] = np.mean((hahn50_nw_res[:, :, 0] - mu)**2, axis=0)
MSEs[11] = np.mean((hahn100_knn_res[:, :, 0] - mu)**2, axis=0)
MSEs[12] = np.mean((hahn100_nw_res[:, :, 0] - mu)**2, axis=0)

STDs[0] = np.std((rct_res[:, :, 0] - mu)**2, axis=0)
STDs[1] = np.std((a2ipw_knn_res[:, :, 0] - mu)**2, axis=0)
STDs[2] = np.std((a2ipw_nw_res[:, :, 0] - mu)**2, axis=0)
STDs[3] = np.std((ma2ipw_knn_res[:, :, 0] - mu)**2, axis=0)
STDs[4] = np.std((ma2ipw_nw_res[:, :, 0] - mu)**2, axis=0)
STDs[5] = np.std((adaipw_knn_res[:, :, 0] - mu)**2, axis=0)
STDs[6] = np.std((adaipw_nw_res[:, :, 0] - mu)**2, axis=0)
STDs[7] = np.std((dm_knn_res[:, :, 0] - mu)**2, axis=0)
STDs[8] = np.std((dm_nw_res[:, :, 0] - mu)**2, axis=0)
STDs[9] = np.std((hahn50_knn_res[:, :, 0] - mu)**2, axis=0)
STDs[10] = np.std((hahn50_nw_res[:, :, 0] - mu)**2, axis=0)
STDs[11] = np.std((hahn100_knn_res[:, :, 0] - mu)**2, axis=0)
STDs[12] = np.std((hahn100_nw_res[:, :, 0] - mu)**2, axis=0)

tau_time[0] = sequential_testing(rct_res, T, delta=delta0)
tau_time[1] = sequential_testing(a2ipw_knn_res, T, delta=delta0)
tau_time[2] = sequential_testing(a2ipw_nw_res, T, delta=delta0)
tau_time[3] = sequential_testing(ma2ipw_knn_res, T, delta=delta0)
tau_time[4] = sequential_testing(ma2ipw_nw_res, T, delta=delta0)
tau_time[5] = sequential_testing(adaipw_knn_res, T, delta=delta0)
tau_time[6] = sequential_testing(adaipw_nw_res, T, delta=delta0)
tau_time[7] = sequential_testing(dm_knn_res, T, delta=delta0)
tau_time[8] = sequential_testing(dm_nw_res, T, delta=delta0)
tau_time[9] = sequential_testing(hahn50_knn_res, T, delta=delta0)
tau_time[10] = sequential_testing(hahn50_nw_res, T, delta=delta0)
tau_time[11] = sequential_testing(hahn100_knn_res, T, delta=delta0)
tau_time[12] = sequential_testing(hahn100_nw_res, T, delta=delta0)

pval_list = t_test(rct_res, T)
rejection250_time[0] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[0] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[0] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(a2ipw_knn_res, T)
rejection250_time[1] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[1] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[1] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(a2ipw_nw_res, T)
rejection250_time[2] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[2] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[2] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(ma2ipw_knn_res, T)
rejection250_time[3] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[3] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[3] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(ma2ipw_nw_res, T)
rejection250_time[4] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[4] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[4] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(adaipw_knn_res, T)
rejection250_time[5] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[5] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[5] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(adaipw_nw_res, T)
rejection250_time[6] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[6] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[6] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(dm_knn_res, T)
rejection250_time[7] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[7] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[7] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(dm_nw_res, T)
rejection250_time[8] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[8] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[8] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(hahn50_knn_res, T)
rejection250_time[9] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[9] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[9] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(hahn50_nw_res, T)
rejection250_time[10] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[10] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[10] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(hahn100_knn_res, T)
rejection250_time[11] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[11] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[11] = multiple_testing(pval_list, alpha=0.05)
pval_list = t_test(hahn100_nw_res, T)
rejection250_time[12] = rejection_or_not(pval_list, alpha=0.05, size=-350)
rejection500_time[12] = rejection_or_not(pval_list, alpha=0.05, size=-200)
bf_time[12] = multiple_testing(pval_list, alpha=0.05)

/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  
/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in greater
/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in log
  if sys.path[0] == '':
/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in sqrt
  if sys.path[0] == '':
/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in true_divide
  del sys.path[0]
/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/s08581/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infra

In [35]:
result_list = [MSEs[:, -350], STDs[:, -350], np.round(rejection250_time*100, 1), MSEs[:, -200], STDs[:, -200], np.round(rejection500_time*100, 1), np.round(tau_time, 1), np.round(bf_time, 1) ]


In [41]:
result_list = [MSEs[:, -350], STDs[:, -350], np.round(rejection250_time*100, 1), MSEs[:, -200], STDs[:, -200], np.round(rejection500_time*100, 1), np.round(tau_time, 1), np.round(bf_time, 1) ]
df = pd.DataFrame(np.round(result_list, 3)).T

In [36]:
print(pd.DataFrame(np.round(result_list, 3)).T.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &      0 &      1 &      2 &      3 &      4 &      5 &      6 &      7 \\
\midrule
0  &  0.674 &  1.066 &   60.4 &  0.333 &  0.562 &   93.4 &  355.4 &  228.0 \\
1  &  0.606 &  0.891 &   99.6 &  0.310 &  0.500 &  100.0 &   86.3 &  150.5 \\
2  &  0.485 &  0.740 &   99.8 &  0.202 &  0.311 &  100.0 &   76.2 &  150.2 \\
3  &  0.599 &  0.961 &   99.5 &  0.275 &  0.432 &  100.0 &   84.6 &  150.5 \\
4  &  0.484 &  0.688 &   99.9 &  0.214 &  0.317 &  100.0 &   74.7 &  150.1 \\
5  &  3.287 &  5.293 &   63.7 &  1.626 &  2.681 &   84.8 &  293.6 &  231.8 \\
6  &  3.694 &  6.056 &   61.5 &  1.770 &  2.896 &   84.7 &  302.6 &  231.1 \\
7  &  1.138 &  1.745 &   99.9 &  0.578 &  0.892 &  100.0 &   15.1 &  150.1 \\
8  &  0.999 &  1.427 &  100.0 &  0.454 &  0.623 &  100.0 &   26.4 &  150.0 \\
9  &  0.725 &  1.164 &   93.7 &  0.320 &  0.491 &  100.0 &  165.9 &  156.7 \\
10 &  0.563 &  0.872 &   95.8 &  0.277 &  0.433 &  100.0 &  154.5 &  154.2 \\
11 &  0.748 &  1.21

In [37]:
result_list = [MSEs[:, -400], STDs[:, -40], MSEs[:, -200], STDs[:, -200], MSEs[:, -350], STDs[:, -200], MSEs[:, -350], STDs[:, -200] ]


In [38]:
print(pd.DataFrame(np.round(result_list, 3)).T.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &      0 &      1 &      2 &      3 &      4 &      5 &      6 &      7 \\
\midrule
0  &  0.968 &  0.341 &  0.333 &  0.562 &  0.674 &  0.562 &  0.674 &  0.562 \\
1  &  0.934 &  0.304 &  0.310 &  0.500 &  0.606 &  0.500 &  0.606 &  0.500 \\
2  &  0.748 &  0.177 &  0.202 &  0.311 &  0.485 &  0.311 &  0.485 &  0.311 \\
3  &  0.955 &  0.280 &  0.275 &  0.432 &  0.599 &  0.432 &  0.599 &  0.432 \\
4  &  0.722 &  0.184 &  0.214 &  0.317 &  0.484 &  0.317 &  0.484 &  0.317 \\
5  &  5.203 &  1.787 &  1.626 &  2.681 &  3.287 &  2.681 &  3.287 &  2.681 \\
6  &  5.242 &  2.189 &  1.770 &  2.896 &  3.694 &  2.896 &  3.694 &  2.896 \\
7  &  1.674 &  0.595 &  0.578 &  0.892 &  1.138 &  0.892 &  1.138 &  0.892 \\
8  &  1.578 &  0.368 &  0.454 &  0.623 &  0.999 &  0.623 &  0.999 &  0.623 \\
9  &  1.222 &  0.301 &  0.320 &  0.491 &  0.725 &  0.491 &  0.725 &  0.491 \\
10 &  0.930 &  0.256 &  0.277 &  0.433 &  0.563 &  0.433 &  0.563 &  0.433 \\
11 &  0.997 &  0.31